In [1]:
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph
from uuid import UUID
from pprint import pprint

### Connect

In [2]:
username = "analyst"
password = "Esri.4.GIS"
url = "https://ebase.ad.local/server/rest/services/Hosted/SupplyChain/KnowledgeGraphServer"
#gis = GIS(r"https://adsrv2019.ad.local/portal", username, password)
gis = GIS(r"https://ebase.ad.local/portal", username=username, password=password)
knowledge_graph = KnowledgeGraph(url=url, gis=gis)
knowledge_graph

### Get list of distributor names to process

In [3]:
distributorNameQuery = """MATCH (p:ProcessingPlant)
WITH DISTINCT p.Distributors as d
RETURN d"""
x=knowledge_graph.query(distributorNameQuery)
y = []
distributorNames = []
for i in range(len(x)):
    y.extend(x[i])
for item in y:
    temp = item.split(", ")
    for s in temp:
        distributorNames.append(s)
distributorNames = list(set(distributorNames))
print(distributorNames)

['Distributor 16', 'Distributor 11', 'Distributor 3', 'Distributor 8', 'Distributor 6', 'Distributor 1', 'Distributor 2', 'Distributor 10', 'Distributor 17', 'Distributor 5', 'Distributor 19', 'Distributor 12', 'Distributor 22', 'Distributor 21', 'Distributor 23', 'Distributor 14', 'Distributor 20', 'Distributor 18', 'Distributor 15', 'Distributor 13', 'Distributor 7', 'Distributor 4', 'Distributor 9']


### Query

In [4]:
def submitQuery(distributorName):
    opQuery = """MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "{0}")
    MATCH (d:Distributor)
    WHERE d.DistributorName = "{0}"
    RETURN p, d""".format(distributorName)
    print(opQuery)

    queryResults = knowledge_graph.query(opQuery)
    return queryResults

### Given returned nodes, create relationships

In [5]:
def createRelationships(queryResults):
    countRels = 0
    for p, d in queryResults:
        print("Related nodes:")
        print(p['_properties']['ProcessingPlantName'])
        #print(p['_id'])
        #print(p['_properties']['Material'])
        print(d['_properties']['DistributorName'])
        print(d['_id'])
        #print(d['_properties']['Material'])
        #print()
        add_relationship = {
            "_objectType": "relationship",
            "_typeName": "SellsTo",
            "_originEntityId": UUID(f"{d['_id']}"), # Distributor ID
            "_destinationEntityId": UUID(f"{p['_id']}") # Plant ID
        }
        knowledge_graph.apply_edits(adds=[add_relationship])
        countRels += 1
        #print("EDITS MADE")
    print(f"All done! {countRels} relationships created.")

### Perform the edits

In [6]:
for distributor in distributorNames:
    print(f"Finding related Processing Plants for {distributor}")
    queryNodes = submitQuery(distributor)
    createRelationships(queryNodes) 
print("End of script.")

Finding related Processing Plants for Distributor 16
MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "Distributor 16")
    MATCH (d:Distributor)
    WHERE d.DistributorName = "Distributor 16"
    RETURN p, d
Related nodes:
Plant 1
Distributor 16
bd20e57a-00e7-4739-b37b-b5e36eb692c0
Related nodes:
Plant 2
Distributor 16
bd20e57a-00e7-4739-b37b-b5e36eb692c0
Related nodes:
Plant 3
Distributor 16
bd20e57a-00e7-4739-b37b-b5e36eb692c0
All done! 3 relationships created.
Finding related Processing Plants for Distributor 11
MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHERE name = "Distributor 11")
    MATCH (d:Distributor)
    WHERE d.DistributorName = "Distributor 11"
    RETURN p, d
Related nodes:
Plant 5
Distributor 11
ffd80e2b-3056-48cd-8c5a-803416464679
All done! 1 relationships created.
Finding related Processing Plants for Distributor 3
MATCH (p:ProcessingPlant)
    WHERE any(name in split(p.Distributors, ", ") WHER